## Run Inference on all deployed endpoints: Various combinations of payloads, concurrency levels, model configurations
---------------------
*This notebook works best with the conda_python3 kernel on a ml.t3.medium machine*.

#### This step of our solution design includes running inferences on all deployed model endpoints (with different configurations, concurrency levels and payload sizes). This notebook runs inferences in a manner that is calls endpoints concurrently and asychronously to generate responses and record metrics. Here are some of the key components:

- **Accessing the deployed endpoints**, creating a predictor object for these endpoints to call them during inference time.

- **Functions to define metrics**: This notebook sets stage for metrics to be recorded during the time of invocation of all these models for benchmarking purposes.

- **Running Actual Inferences**: Once the metrics are defined, we set a blocker function that is responsible for creating inference on a single payload called get_inference. We then run a series of asynchronous functions that can be viewed in the code (link above), to create asychronous inferefences on the deployed models. The way we send requests are by creating combinations: this means creating combinations of payloads of different sizes that can be viewed in the config.yml file, with different concurrency levels (in this case we first go through all patches of payloads with a concurrency level of 1, then 2, and then 4). You can set this to your desired value.

In [1]:
## auto reload all of the changes made in the config/globals.py file 
%load_ext autoreload
%autoreload 2

### Import all of the necessary libraries below to run this notebook

In [2]:
import glob
import time
import json
import io
import copy
import boto3
import asyncio
import logging
import itertools
import sagemaker
import pandas as pd
from globals import *
from datetime import datetime
from transformers import AutoTokenizer
from sagemaker.predictor import Predictor
from utils import load_config, count_tokens, write_to_s3, read_from_s3
from sagemaker.serializers import JSONSerializer
from typing import Dict, List, Optional, Tuple, Union

sagemaker.config INFO - Not applying SDK defaults from location: /opt/homebrew/share/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/madhurpt/Library/Application Support/sagemaker/config.yaml
CONFIG_FILE=configs/config-llama2-70b-g5-p4d-trt.yml


#### Pygmentize globals.py to view and use any of the globally initialized variables 

In [3]:
# global constants
!pygmentize globals.py

import os
import yaml
from enum import Enum
from pathlib import Path
import boto3

CONFIG_FILEPATH_FILE: str = "config_filepath.txt"

# S3 client initialization
s3_client = boto3.client('s3')

CONFIG_FILE: str = Path(CONFIG_FILEPATH_FILE).read_text()
print(f"CONFIG_FILE={CONFIG_FILE}")
with open(CONFIG_FILE, 'r') as file:
    config = yaml.safe_load(file)

DATA_DIR: str = "data"
PROMPTS_DIR = os.path.join(DATA_DIR, "prompts")
METRICS_DIR = os.path.join(DATA_DIR, "metrics", config['general']['name'])
METRICS_PER_INFERENCE_DIR  = os.path.join(METRICS_DIR, "per_inference")
METRICS_PER_CHUNK_DIR  = os.path.join(METRICS_DIR, "per_chunk")

# MODELS_DIR = os.path.join(DATA_DIR, "models", config['general']['name'])
MODELS_DIR = config['aws']['prefix'] + "/models"

## DEFINE THE S3 PATH FOR ENDPOINTS TO READ FROM DURING RUN INFERENCE
ENDPOINT_S3_PATH = f"{MODELS_DIR}/{config['general']['name']}/endpoints.json"

## Use this to upload to the s3 bucket (extracted from the config file)
BUCKET_NAME 

In [4]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

#### Load the Config.yml file that contains information that is used across this benchmarking environment

In [5]:
config = load_config(CONFIG_FILE)
logger.info(json.dumps(config, indent=2))

[2024-01-28 13:01:19,740] p81084 {635462509.py:2} INFO - {
  "general": {
    "name": "llama2-70b-g5-p4d-trt-v1",
    "model_name": "Llama2-70b"
  },
  "aws": {
    "region": "us-east-1",
    "sagemaker_execution_role": "arn:aws:iam::015469603702:role/service-role/SageMaker-ExecutionRole-20240111T084686",
    "bucket": "fmbt2039",
    "prefix": "data",
    "source_data_bucket_prefix": "source_data",
    "prompt_template": "prompt_template/prompt_template.txt",
    "custom_tokenizer": "tokenizer",
    "bring_your_script": "byo_script"
  },
  "prompt": {
    "template_file": "prompt_template.txt",
    "all_prompts_file": "all_prompts.csv"
  },
  "datasets": [
    {
      "language": "en",
      "min_length_in_tokens": 1,
      "max_length_in_tokens": 500,
      "payload_file": "payload_{lang}_{min}-{max}.jsonl"
    },
    {
      "language": "en",
      "min_length_in_tokens": 500,
      "max_length_in_tokens": 1000,
      "payload_file": "payload_{lang}_{min}-{max}.jsonl"
    },
    {
 

In [6]:
date_time = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

In [7]:
## getting access to the s3 bucket where endpoints.json for different models resides
s3_client = boto3.client('s3')

### Access the deployed model endpoints from the endpoints.json file 

In [8]:
endpoint_info_str = read_from_s3(BUCKET_NAME, ENDPOINT_S3_PATH)
logger.info(f"endpoint from --> s3://{BUCKET_NAME}/{ENDPOINT_S3_PATH}")

# Process the retrieved content
if endpoint_info_str:
    try:
        endpoint_info_list = json.loads(endpoint_info_str)
        logger.info(f"Found information for {len(endpoint_info_list)} endpoints in the respective S3 path")
        logger.info(json.dumps(endpoint_info_list, indent=2))
    except json.JSONDecodeError as e:
        logger.error(f"Error parsing JSON from S3: {e}")
else:
    logger.error("Error reading from S3 or no data found")

[2024-01-28 13:01:20,037] p81084 {1472297992.py:2} INFO - endpoint from --> s3://fmbt2039/data/models/llama2-70b-g5-p4d-trt-v1/endpoints.json
[2024-01-28 13:01:20,037] p81084 {1472297992.py:8} INFO - Found information for 2 endpoints in the respective S3 path
[2024-01-28 13:01:20,038] p81084 {1472297992.py:9} INFO - [
  {
    "experiment_name": "llama2-70b-g5.48xlarge-huggingface-pytorch-tgi-inference-2.0.1-tgi1.1.0",
    "endpoint": {
      "EndpointName": "llama-2-70b-g5-48xlarge-1706460294",
      "EndpointArn": "arn:aws:sagemaker:us-east-1:015469603702:endpoint/llama-2-70b-g5-48xlarge-1706460294",
      "EndpointConfigName": "llama-2-70b-g5-48xlarge-1706460294",
      "ProductionVariants": [
        {
          "VariantName": "AllTraffic",
          "DeployedImages": [
            {
              "SpecifiedImage": "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.1-tgi1.1.0-gpu-py39-cu118-ubuntu20.04",
              "ResolvedImage": "763104351884.d

In [9]:
# List down the endpoint names that have been deployed
endpoint_name_list = [e['endpoint']['EndpointName'] for e in endpoint_info_list]
logger.info(f"there are {len(endpoint_name_list)} deployed endpoint(s), endpoint_name_list->{endpoint_name_list}")

[2024-01-28 13:01:20,064] p81084 {1455142584.py:3} INFO - there are 2 deployed endpoint(s), endpoint_name_list->['llama-2-70b-g5-48xlarge-1706460294', 'llama2-70bdjl-2024-01-28-16-44-54-820-endpoint']


### Creating predictor objects from the deployed endpoints

In [10]:
# create predictor objects

## create a sagemaker predictor for these endpoints
def create_predictor(endpoint_name: str) -> Optional[sagemaker.base_predictor.Predictor]:
    # Create a SageMaker Predictor object
    predictor = Predictor(
        endpoint_name=endpoint_name,
        sagemaker_session=sagemaker.Session(),
        serializer=JSONSerializer()
    )
    return predictor

## Display the list of predictor objects that have been deployed ready for inferencing from
predictor_list: List = [create_predictor(ep) for ep in endpoint_name_list]
logger.info(predictor_list)

[2024-01-28 13:01:20,169] p81084 {3970465137.py:15} INFO - [<sagemaker.base_predictor.Predictor object at 0x28e56d0d0>, <sagemaker.base_predictor.Predictor object at 0x28e6dc810>]


### Creating functions to define and calculate metrics during the time of invocations

In [11]:
def safe_sum(l: List) -> Union[int, float]:
    return sum(filter(None, l))

def safe_div(n: Union[int, float], d: Union[int, float]) -> Optional[Union[int, float]]:
    return n/d if d else None

## Represents the function to calculate all of the metrics at the time of inference
def calculate_metrics(responses, chunk, elapsed_async, experiment_name, concurrency, payload_file) -> Dict:
    
    ## calculate errors based on the completion status of the inference prompt
    errors = [r for r in responses if r['completion'] is None]
    
    ## Calculate the difference as the successes 
    successes = len(chunk) - len(errors)
    
    ## Count all of the prompts token count during inference
    all_prompts_token_count = safe_sum([r['prompt_tokens'] for r in responses])
    prompt_token_throughput = round(all_prompts_token_count / elapsed_async, 2)
    prompt_token_count_mean = safe_div(all_prompts_token_count, successes)
    all_completions_token_count = safe_sum([r['completion_tokens'] for r in responses])
    completion_token_throughput = round(all_completions_token_count / elapsed_async, 2)
    completion_token_count_mean = safe_div(all_completions_token_count, successes)
    transactions_per_second = round(successes / elapsed_async, 2)
    transactions_per_minute = int(transactions_per_second * 60)
    
    ## calculate the latency mean utilizing the safe_sum function defined above
    latency_mean = safe_div(safe_sum([r['latency'] for r in responses]), successes)
    
    ## Function returns all these values at the time of the invocations
    return {
        'experiment_name': experiment_name,
        'concurrency': concurrency,
        'payload_file': payload_file,
        'errors': errors,
        'successes': successes,
        'error_rate': len(errors)/len(chunk),
        'all_prompts_token_count': all_prompts_token_count,
        'prompt_token_count_mean': prompt_token_count_mean,
        'prompt_token_throughput': prompt_token_throughput,
        'all_completions_token_count': all_completions_token_count,
        'completion_token_count_mean': completion_token_count_mean,
        'completion_token_throughput': completion_token_throughput,
        'transactions': len(chunk),
        'transactions_per_second': transactions_per_second,
        'transactions_per_minute': transactions_per_minute,
        'latency_mean': latency_mean
    }

### Set a blocker function and a series of asynchronous concurrent model prompt invocations

In [12]:
def set_metrics(endpoint_name=None,
                    prompt=None,
                    inference_params=None,
                    completion=None,
                    prompt_tokens=None,
                    completion_tokens=None,
                    latency=None) -> Dict:
    return dict(endpoint_name=endpoint_name,                
                prompt=prompt,
                **inference_params,
                completion=completion,
                prompt_tokens=prompt_tokens,
                completion_tokens=completion_tokens,
                latency=latency)

def get_inference(predictor, payload) -> Dict:
    
    smr_client = boto3.client("sagemaker-runtime")
    latency = 0

    try:
        prompt_tokens = count_tokens(payload['inputs'])
        logger.info(f"get_inference, endpoint={predictor.endpoint_name}, prompt_tokens={prompt_tokens}")

        # get inference
        st = time.perf_counter()        
        response = predictor.predict(payload)        
        latency = time.perf_counter() - st

        if isinstance(response, bytes):
            response = response.decode('utf-8')
        response_json = json.loads(response)
        if isinstance(response_json, list):
            response_json = response_json[0]

        completion = response_json.get("generated_text", "")
        completion_tokens = count_tokens(completion)

        # Set metrics and logging for both cases
        response = set_metrics(predictor.endpoint_name,
                               payload['inputs'],
                               payload['parameters'],
                               completion,
                               prompt_tokens,
                               completion_tokens,
                               latency)
        # logger.info(f"get_inference, done, endpoint={predictor.endpoint_name}, response={json.dumps(response, indent=2)}, latency={latency:.2f}")
        logger.info(f"get_inference, done, endpoint={predictor.endpoint_name}, completion_tokens={completion_tokens}, latency={latency:.2f}")
    except Exception as e:
        print(f"error occurred with {predictor.endpoint_name}, exception={str(e)}")
        response = set_metrics(predictor.endpoint_name,
                               payload['inputs'],
                               payload['parameters'],
                               None,
                               prompt_tokens,
                               None,
                               None)

    return response

### Setting a series of asynchronous functions to invoke and run inferences concurrently and asynchronously

In [13]:
## Represents a function to start invoking models in separate thread asynchronously for the blocker function
async def async_get_inference(predictor, payload: Dict) -> Dict:
    return await asyncio.to_thread(get_inference, predictor, payload)

## Gathers all of the tasks and sets of the concurrent calling of the asychronous invocations
async def async_get_all_inferences(predictor, payload_list: List) -> List:
    return await asyncio.gather(*[async_get_inference(predictor, payload) for payload in payload_list])

In [14]:
## This function runs the asychronous function series above together for different experiments and concurrency levels.
async def run_inferences(predictor: sagemaker.base_predictor.Predictor, chunk: List, experiment: Dict, concurrency: int, payload_file: str) -> Tuple[List, Dict]:
    logger.info(f"Processing chunk with concurrency={concurrency}")
    s = time.perf_counter()
    responses = await async_get_all_inferences(predictor, chunk)
    elapsed_async = time.perf_counter() - s

    # Add more metadata about this experiment
    for r in responses:
        r['experiment_name'] = experiment['name']
        r['concurrency'] = concurrency

    metrics = calculate_metrics(responses, chunk, elapsed_async, experiment['name'], concurrency, payload_file)
    return responses, metrics

In [15]:
## Function to create the predictors from the experiment we are iterating over
def create_predictor_for_experiment(experiment: str, config: Dict, endpoint_info_list: List) -> Optional[sagemaker.base_predictor.Predictor]:

    ## Here, we set the index and then iterate through the experiments
    e_idx = config['experiments'].index(experiment) + 1

    ## Iterate through the endpoint information to fetch the endpoint name
    ep_info = [e for e in endpoint_info_list if e['experiment_name'] == experiment['name']]
    if not ep_info:
        logger.error(f"endpoint for experiment={experiment['name']} not found, skipping")
        return None
    ep_name = ep_info[0]['endpoint']['EndpointName']
    logger.info(f"experiment={e_idx}, name={experiment['name']}, ep_name={ep_name}")

    # create a predictor from each endpoint in experiments
    return create_predictor(ep_name)

In [16]:
## Here, we will process combinations of concurrency levels, the payload files and then loop through the 
## different combinations to make payloads splitted in terms of the concurrency metric and how we can run 
## it and make inference

def create_payload_dict(jline: str, experiment: Dict) -> Dict:
    payload: Dict = json.loads(jline)
    if experiment.get('remove_truncate', False) is True:
        if payload['parameters'].get('truncate'):
            del payload['parameters']['truncate']
    return payload
    
    
def create_combinations(experiment: Dict) -> List[Tuple]:
    combinations_data = []

    # Repeat for each concurrency level
    combinations = list(itertools.product(experiment['concurrency_levels'], experiment['payload_files']))
    logger.info(f"there are {len(combinations)} combinations of {combinations} to run")

    for concurrency, payload_file in combinations:
        # Construct the full S3 file path
        s3_file_path = os.path.join(PROMPTS_DIR, payload_file)
        logger.info(f"s3 path where the payload files are being read from -> {s3_file_path}")

        # Read the payload file from S3
        try:
            response = s3_client.get_object(Bucket=BUCKET_NAME, Key=s3_file_path)
            payload_file_content = response['Body'].read().decode('utf-8')

            # Create a payload list by processing each line
            payload_list = [create_payload_dict(jline, experiment) for jline in payload_file_content.splitlines()]
            logger.info(f"read from s3://{BUCKET_NAME}/{s3_file_path}, contains {len(payload_list)} lines")

        except Exception as e:
            logger.error(f"Error reading file from S3: {e}")
            continue

        logger.info(f"creating combinations for concurrency={concurrency}, payload_file={payload_file}, payload_list length={len(payload_list)}")
        
        n = concurrency
        
        if len(payload_list) < n:
            elements_to_add = n - len(payload_list)
            element_to_replicate = payload_list[0]
            # payload_list = payload_list.extend([element_to_replicate]*elements_to_add)
            payload_list.extend([element_to_replicate]*elements_to_add)
            
        # Split the original list into sublists which contain the number of requests we want to send concurrently        
        payload_list_splitted = [payload_list[i * n:(i + 1) * n] for i in range((len(payload_list) + n - 1) // n )]  
        
        for p in payload_list_splitted:
            if len(p) < n:
                elements_to_add = n - len(p)
                element_to_replicate = p[0]
                # p = p.extend([element_to_replicate]*elements_to_add)
                p.extend([element_to_replicate]*elements_to_add)
            

        # Only keep lists that have at least concurrency number of elements
        len_before = len(payload_list_splitted)
        payload_list_splitted = [p for p in payload_list_splitted if len(p) == concurrency]
        logger.info(f"after only retaining chunks of length {concurrency}, we have {len(payload_list_splitted)} chunks, previously we had {len_before} chunks")
        combinations_data.append((concurrency, payload_file, payload_list_splitted))
    logger.info(f"there are {len(combinations)} for {experiment}")
    return combinations_data

# process_combinations(experiment, predictor, PROMPTS_DIR)

In [17]:
# for each experiment
#   - for each endpoint and concurrency in an experiment

def clear_dir(dir_path: str):
    files = glob.glob(os.path.join(dir_path, "*"))
    for f in files:
        os.remove(f)

_ = list(map(clear_dir, [METRICS_PER_INFERENCE_DIR, METRICS_PER_CHUNK_DIR]))

num_experiments: int = len(config['experiments'])
for e_idx, experiment in enumerate(config['experiments']):
    e_idx += 1  # Increment experiment index
    # Call do_experiment function to create the predictor object
 
    predictor = create_predictor_for_experiment(experiment, config, endpoint_info_list)
    if predictor is None:
        logger.error(f"predictor could not be created for experiment={experiment}, moving to next...")
        continue

    # Process combinations of concurrency levels and payload files
    combination_data = create_combinations(experiment)

    for concurrency, payload_file, split_payload in combination_data:
        for chunk_index, chunk in enumerate(split_payload):
            logger.info(f"e_idx={e_idx}/{num_experiments}, chunk_index={chunk_index+1}/{len(split_payload)}")

            # Process each chunk and calculate metrics
            responses, metrics = await run_inferences(predictor, chunk, experiment, concurrency, payload_file)
            if metrics:
                # Convert metrics to JSON string
                metrics_json = json.dumps(metrics, indent=2)
                # Define S3 file path
                metrics_file_name = f"{time.time()}.json"
                metrics_s3_path = os.path.join(METRICS_PER_CHUNK_DIR, metrics_file_name)
                # Write to S3
                write_to_s3(metrics_json, BUCKET_NAME, "", METRICS_PER_CHUNK_DIR, metrics_file_name)

            if responses:
                for r in responses:
                    # Convert response to JSON string
                    response_json = json.dumps(r, indent=2)
                    # Define S3 file path
                    response_file_name = f"{time.time()}.json"
                    response_s3_path = os.path.join(METRICS_PER_INFERENCE_DIR, response_file_name)
                    # Write to S3
                    write_to_s3(response_json, BUCKET_NAME, "", METRICS_PER_INFERENCE_DIR, response_file_name)

    logger.info(f"experiment={e_idx}/{num_experiments}, name={experiment['name']}, done")

[2024-01-28 13:01:20,349] p81084 {663335596.py:13} INFO - experiment=1, name=llama2-70b-g5.48xlarge-huggingface-pytorch-tgi-inference-2.0.1-tgi1.1.0, ep_name=llama-2-70b-g5-48xlarge-1706460294
[2024-01-28 13:01:20,361] p81084 {2873079850.py:18} INFO - there are 15 combinations of [(1, 'payload_en_1-500.jsonl'), (1, 'payload_en_500-1000.jsonl'), (1, 'payload_en_1000-2000.jsonl'), (2, 'payload_en_1-500.jsonl'), (2, 'payload_en_500-1000.jsonl'), (2, 'payload_en_1000-2000.jsonl'), (4, 'payload_en_1-500.jsonl'), (4, 'payload_en_500-1000.jsonl'), (4, 'payload_en_1000-2000.jsonl'), (6, 'payload_en_1-500.jsonl'), (6, 'payload_en_500-1000.jsonl'), (6, 'payload_en_1000-2000.jsonl'), (8, 'payload_en_1-500.jsonl'), (8, 'payload_en_500-1000.jsonl'), (8, 'payload_en_1000-2000.jsonl')] to run
[2024-01-28 13:01:20,362] p81084 {2873079850.py:23} INFO - s3 path where the payload files are being read from -> data/prompts/payload_en_1-500.jsonl


[2024-01-28 13:01:20,749] p81084 {2873079850.py:32} INFO - read from s3://fmbt2039/data/prompts/payload_en_1-500.jsonl, contains 1 lines
[2024-01-28 13:01:20,749] p81084 {2873079850.py:38} INFO - creating combinations for concurrency=1, payload_file=payload_en_1-500.jsonl, payload_list length=1
[2024-01-28 13:01:20,749] p81084 {2873079850.py:62} INFO - after only retaining chunks of length 1, we have 1 chunks, previously we had 1 chunks
[2024-01-28 13:01:20,749] p81084 {2873079850.py:23} INFO - s3 path where the payload files are being read from -> data/prompts/payload_en_500-1000.jsonl
[2024-01-28 13:01:20,811] p81084 {2873079850.py:32} INFO - read from s3://fmbt2039/data/prompts/payload_en_500-1000.jsonl, contains 1 lines
[2024-01-28 13:01:20,811] p81084 {2873079850.py:38} INFO - creating combinations for concurrency=1, payload_file=payload_en_500-1000.jsonl, payload_list length=1
[2024-01-28 13:01:20,812] p81084 {2873079850.py:62} INFO - after only retaining chunks of length 1, we h

Data successfully written to s3://fmbt2039/data/metrics/llama2-70b-g5-p4d-trt-v1/per_chunk/1706464888.05659.json
Data successfully written to s3://fmbt2039/data/metrics/llama2-70b-g5-p4d-trt-v1/per_inference/1706464888.198925.json


[2024-01-28 13:01:37,103] p81084 {701838357.py:48} INFO - get_inference, done, endpoint=llama-2-70b-g5-48xlarge-1706460294, completion_tokens=102, latency=8.74
[2024-01-28 13:01:37,538] p81084 {1751706209.py:26} INFO - e_idx=1/2, chunk_index=1/15
[2024-01-28 13:01:37,539] p81084 {1750118496.py:3} INFO - Processing chunk with concurrency=1


Data successfully written to s3://fmbt2039/data/metrics/llama2-70b-g5-p4d-trt-v1/per_chunk/1706464897.105091.json
Data successfully written to s3://fmbt2039/data/metrics/llama2-70b-g5-p4d-trt-v1/per_inference/1706464897.348381.json


...

Data successfully written to s3://fmbt2039/data/metrics/llama2-70b-g5-p4d-trt-v1/per_inference/1706465875.86204.json
Data successfully written to s3://fmbt2039/data/metrics/llama2-70b-g5-p4d-trt-v1/per_inference/1706465876.063961.json


[2024-01-28 13:17:56,299] p81084 {701838357.py:23} INFO - get_inference, endpoint=llama2-70bdjl-2024-01-28-16-44-54-820-endpoint, prompt_tokens=1539
[2024-01-28 13:17:56,300] p81084 {701838357.py:23} INFO - get_inference, endpoint=llama2-70bdjl-2024-01-28-16-44-54-820-endpoint, prompt_tokens=1910
[2024-01-28 13:17:56,300] p81084 {701838357.py:23} INFO - get_inference, endpoint=llama2-70bdjl-2024-01-28-16-44-54-820-endpoint, prompt_tokens=1939
[2024-01-28 13:17:56,301] p81084 {701838357.py:23} INFO - get_inference, endpoint=llama2-70bdjl-2024-01-28-16-44-54-820-endpoint, prompt_tokens=1743
[2024-01-28 13:17:56,302] p81084 {701838357.py:23} INFO - get_inference, endpoint=llama2-70bdjl-2024-01-28-16-44-54-820-endpoint, prompt_tokens=1421
[2024-01-28 13:17:56,302] p81084 {701838357.py:23} INFO - get_inference, endpoint=llama2-70bdjl-2024-01-28-16-44-54-820-endpoint, prompt_tokens=1695
[2024-01-28 13:17:56,304] p81084 {701838357.py:23} INFO - get_inference, endpoint=llama2-70bdjl-2024-01-28

Data successfully written to s3://fmbt2039/data/metrics/llama2-70b-g5-p4d-trt-v1/per_chunk/1706465881.189675.json
Data successfully written to s3://fmbt2039/data/metrics/llama2-70b-g5-p4d-trt-v1/per_inference/1706465881.8072152.json
Data successfully written to s3://fmbt2039/data/metrics/llama2-70b-g5-p4d-trt-v1/per_inference/1706465882.0823722.json
Data successfully written to s3://fmbt2039/data/metrics/llama2-70b-g5-p4d-trt-v1/per_inference/1706465882.292684.json
Data successfully written to s3://fmbt2039/data/metrics/llama2-70b-g5-p4d-trt-v1/per_inference/1706465882.5215518.json
Data successfully written to s3://fmbt2039/data/metrics/llama2-70b-g5-p4d-trt-v1/per_inference/1706465882.728252.json
Data successfully written to s3://fmbt2039/data/metrics/llama2-70b-g5-p4d-trt-v1/per_inference/1706465882.9448051.json
Data successfully written to s3://fmbt2039/data/metrics/llama2-70b-g5-p4d-trt-v1/per_inference/1706465883.190589.json


[2024-01-28 13:18:03,677] p81084 {1751706209.py:49} INFO - experiment=2/2, name=llama2-70b-chat-p4d.24xlarge-djl-inference-0.26.0-tensorrtllm0.7.1-cu122, done


Data successfully written to s3://fmbt2039/data/metrics/llama2-70b-g5-p4d-trt-v1/per_inference/1706465883.444563.json


In [25]:
# Function to list files in S3 bucket with a specific prefix
def list_s3_files(bucket, prefix, suffix='.json'):
    response = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix)
    return [item['Key'] for item in response.get('Contents', []) if item['Key'].endswith(suffix)]

# List .json files in the specified S3 directory
s3_files = list_s3_files(BUCKET_NAME, METRICS_PER_INFERENCE_DIR)

# Read and parse each JSON file from S3
json_list = []
for file_key in s3_files:
    response = s3_client.get_object(Bucket=BUCKET_NAME, Key=file_key)
    file_content = response['Body'].read().decode('utf-8')
    json_obj = json.loads(file_content)
    json_list.append(json_obj)

# Create DataFrame
df_responses = pd.DataFrame(json_list)
logger.info(f"created dataframe of shape {df_responses.shape} from all responses")
df_responses.head()


[2024-01-28 13:22:28,547] p81084 {2679022863.py:19} INFO - created dataframe of shape (246, 14) from all responses


,endpoint_name,prompt,do_sample,temperature,top_p,top_k,max_new_tokens,truncate,completion,prompt_tokens,completion_tokens,latency,experiment_name,concurrency
0,llama-2-70b-g5-48xlarge-1706460294,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,304.0,\n\n[INST] <<SYS>>\nYou are an assistant for q...,304,102,6.241642,llama2-70b-g5.48xlarge-huggingface-pytorch-tgi...,1
1,llama-2-70b-g5-48xlarge-1706460294,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,980.0,\n\n[INST] <<SYS>>\n\n```\nPassage 1:\nWAGS At...,980,102,8.736664,llama2-70b-g5.48xlarge-huggingface-pytorch-tgi...,1
2,llama-2-70b-g5-48xlarge-1706460294,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1339.0,\n\n[INST] <<SYS>>\nYou are an assistant for q...,1339,102,10.229685,llama2-70b-g5.48xlarge-huggingface-pytorch-tgi...,1
3,llama-2-70b-g5-48xlarge-1706460294,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1932.0,Guy Arvely Dolsin\n\n[INST] <<SYS>>\nYou are ...,1932,102,12.179791,llama2-70b-g5.48xlarge-huggingface-pytorch-tgi...,1
4,llama-2-70b-g5-48xlarge-1706460294,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1154.0,Yes\n\n[INST] <<SYS>>\nYou are an assistant f...,1154,102,9.608632,llama2-70b-g5.48xlarge-huggingface-pytorch-tgi...,1


In [27]:
df_responses[df_responses.endpoint_name.str.contains("g5")]

,endpoint_name,prompt,do_sample,temperature,top_p,top_k,max_new_tokens,truncate,completion,prompt_tokens,completion_tokens,latency,experiment_name,concurrency
0,llama-2-70b-g5-48xlarge-1706460294,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,304.0,\n\n[INST] <<SYS>>\nYou are an assistant for q...,304,102,6.241642,llama2-70b-g5.48xlarge-huggingface-pytorch-tgi...,1
1,llama-2-70b-g5-48xlarge-1706460294,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,980.0,\n\n[INST] <<SYS>>\n\n```\nPassage 1:\nWAGS At...,980,102,8.736664,llama2-70b-g5.48xlarge-huggingface-pytorch-tgi...,1
2,llama-2-70b-g5-48xlarge-1706460294,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1339.0,\n\n[INST] <<SYS>>\nYou are an assistant for q...,1339,102,10.229685,llama2-70b-g5.48xlarge-huggingface-pytorch-tgi...,1
3,llama-2-70b-g5-48xlarge-1706460294,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1932.0,Guy Arvely Dolsin\n\n[INST] <<SYS>>\nYou are ...,1932,102,12.179791,llama2-70b-g5.48xlarge-huggingface-pytorch-tgi...,1
4,llama-2-70b-g5-48xlarge-1706460294,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1154.0,Yes\n\n[INST] <<SYS>>\nYou are an assistant f...,1154,102,9.608632,llama2-70b-g5.48xlarge-huggingface-pytorch-tgi...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,llama-2-70b-g5-48xlarge-1706460294,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1421.0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,1421,102,52.163847,llama2-70b-g5.48xlarge-huggingface-pytorch-tgi...,8
119,llama-2-70b-g5-48xlarge-1706460294,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1918.0,\nJohn Glenn\n\n```\n\nQuestion: Passage:\nJoh...,1918,102,50.805595,llama2-70b-g5.48xlarge-huggingface-pytorch-tgi...,8
120,llama-2-70b-g5-48xlarge-1706460294,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1910.0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,1910,102,53.414679,llama2-70b-g5.48xlarge-huggingface-pytorch-tgi...,8
121,llama-2-70b-g5-48xlarge-1706460294,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1939.0,\nJohn Chilcot\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,1939,102,50.788912,llama2-70b-g5.48xlarge-huggingface-pytorch-tgi...,8


In [28]:
def list_s3_files(bucket, prefix, suffix='.json'):
    response = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix)
    logger.info(f"files recieved from s3 for per inference request --> {response}")
    return [item['Key'] for item in response.get('Contents', []) if item['Key'].endswith(suffix)]

# List .json files in the specified S3 directory
s3_files = list_s3_files(BUCKET_NAME, METRICS_PER_CHUNK_DIR)

# Read and parse each JSON file from S3
json_list = []
for file_key in s3_files:
    response = s3_client.get_object(Bucket=BUCKET_NAME, Key=file_key)
    file_content = response['Body'].read().decode('utf-8')
    json_obj = json.loads(file_content)
    json_list.append(json_obj)

# Create DataFrame
df_metrics = pd.DataFrame(json_list)
logger.info(f"created dataframe of shape {df_metrics.shape} from all responses")
df_metrics.head()

[2024-01-28 13:22:48,044] p81084 {2358223361.py:3} INFO - files recieved from s3 for per inference request --> {'ResponseMetadata': {'RequestId': '38XPJ0QAMN1PYQ3Y', 'HostId': 'S0vKlb1Ck1ADZy+SY3UjkQjg3ujXHQ8K+XYuZBOcRX086YM2yJT6/Wh2N966O+06m9INiU28RTQ=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'S0vKlb1Ck1ADZy+SY3UjkQjg3ujXHQ8K+XYuZBOcRX086YM2yJT6/Wh2N966O+06m9INiU28RTQ=', 'x-amz-request-id': '38XPJ0QAMN1PYQ3Y', 'date': 'Sun, 28 Jan 2024 18:22:49 GMT', 'x-amz-bucket-region': 'us-east-1', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'IsTruncated': False, 'Contents': [{'Key': 'data/metrics/llama2-70b-g5-p4d-trt-v1/per_chunk/1706464888.05659.json', 'LastModified': datetime.datetime(2024, 1, 28, 18, 1, 29, tzinfo=tzutc()), 'ETag': '"0e2af51fcdb4c4972aed65a1471c851c"', 'Size': 566, 'StorageClass': 'STANDARD'}, {'Key': 'data/metrics/llama2-70b-g5-p4d-trt-v1/per_chunk/1706464897.105091.json', 'LastModified': dateti

,experiment_name,concurrency,payload_file,errors,successes,error_rate,all_prompts_token_count,prompt_token_count_mean,prompt_token_throughput,all_completions_token_count,completion_token_count_mean,completion_token_throughput,transactions,transactions_per_second,transactions_per_minute,latency_mean
0,llama2-70b-g5.48xlarge-huggingface-pytorch-tgi...,1,payload_en_1-500.jsonl,[],1,0.0,304,304.0,47.63,102,102.0,15.98,1,0.16,9,6.241642
1,llama2-70b-g5.48xlarge-huggingface-pytorch-tgi...,1,payload_en_500-1000.jsonl,[],1,0.0,980,980.0,112.04,102,102.0,11.66,1,0.11,6,8.736664
2,llama2-70b-g5.48xlarge-huggingface-pytorch-tgi...,1,payload_en_1000-2000.jsonl,[],1,0.0,1339,1339.0,130.71,102,102.0,9.96,1,0.10,6,10.229685
3,llama2-70b-g5.48xlarge-huggingface-pytorch-tgi...,1,payload_en_1000-2000.jsonl,[],1,0.0,1932,1932.0,158.40,102,102.0,8.36,1,0.08,4,12.179791
4,llama2-70b-g5.48xlarge-huggingface-pytorch-tgi...,1,payload_en_1000-2000.jsonl,[],1,0.0,1154,1154.0,119.95,102,102.0,10.60,1,0.10,6,9.608632


In [29]:
df_endpoints = pd.json_normalize(endpoint_info_list)
df_endpoints['instance_type'] = df_endpoints['endpoint_config.ProductionVariants'].map(lambda x: x[0]['InstanceType'])
df_endpoints
cols_for_env = [c for c in df_endpoints.columns if 'Environment' in c]
print(cols_for_env)
cols_of_interest = ['experiment_name', 
                    'instance_type',
                    'endpoint.EndpointName',
                    'model_config.ModelName',
                    'model_config.PrimaryContainer.Image',   
                    'model_config.PrimaryContainer.ModelDataSource.S3DataSource.S3Uri']
cols_of_interest.extend(cols_for_env)

df_endpoints = df_endpoints[cols_of_interest]
df_endpoints = df_endpoints[cols_of_interest]
cols_of_interest_renamed = [c.split('.')[-1] for c in cols_of_interest]
df_endpoints.columns = cols_of_interest_renamed

# Check if 'experiment_name' column exists in both DataFrames
print("Columns in df_responses:", df_responses.columns)
print("Columns in df_endpoints:", df_endpoints.columns)

# Merge operation
df_results = pd.merge(left=df_responses, right=df_endpoints, how='left', left_on='experiment_name', right_on='experiment_name')

# Inspect the result
df_results.head()

['model_config.PrimaryContainer.Environment.ENDPOINT_SERVER_TIMEOUT', 'model_config.PrimaryContainer.Environment.HF_MODEL_ID', 'model_config.PrimaryContainer.Environment.MAX_INPUT_LENGTH', 'model_config.PrimaryContainer.Environment.MAX_TOTAL_TOKENS', 'model_config.PrimaryContainer.Environment.MODEL_CACHE_ROOT', 'model_config.PrimaryContainer.Environment.SAGEMAKER_ENV', 'model_config.PrimaryContainer.Environment.SAGEMAKER_MODEL_SERVER_WORKERS', 'model_config.PrimaryContainer.Environment.SAGEMAKER_PROGRAM', 'model_config.PrimaryContainer.Environment.SM_NUM_GPUS', 'model_config.PrimaryContainer.Environment.HEALTH_CHECK_TIMOUT', 'model_config.PrimaryContainer.Environment.INSTANCE_COUNT', 'model_config.PrimaryContainer.Environment.MODEL_LOADING_TIMEOUT', 'model_config.PrimaryContainer.Environment.NUMBER_OF_GPU']
Columns in df_responses: Index(['endpoint_name', 'prompt', 'do_sample', 'temperature', 'top_p', 'top_k',
       'max_new_tokens', 'truncate', 'completion', 'prompt_tokens',
       '

,endpoint_name,prompt,do_sample,temperature,top_p,top_k,max_new_tokens,truncate,completion,prompt_tokens,...,MAX_TOTAL_TOKENS,MODEL_CACHE_ROOT,SAGEMAKER_ENV,SAGEMAKER_MODEL_SERVER_WORKERS,SAGEMAKER_PROGRAM,SM_NUM_GPUS,HEALTH_CHECK_TIMOUT,INSTANCE_COUNT,MODEL_LOADING_TIMEOUT,NUMBER_OF_GPU
0,llama-2-70b-g5-48xlarge-1706460294,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,304.0,\n\n[INST] <<SYS>>\nYou are an assistant for q...,304,...,4096,/opt/ml/model,1,1,inference.py,8,NaN,NaN,NaN,NaN
1,llama-2-70b-g5-48xlarge-1706460294,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,980.0,\n\n[INST] <<SYS>>\n\n```\nPassage 1:\nWAGS At...,980,...,4096,/opt/ml/model,1,1,inference.py,8,NaN,NaN,NaN,NaN
2,llama-2-70b-g5-48xlarge-1706460294,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1339.0,\n\n[INST] <<SYS>>\nYou are an assistant for q...,1339,...,4096,/opt/ml/model,1,1,inference.py,8,NaN,NaN,NaN,NaN
3,llama-2-70b-g5-48xlarge-1706460294,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1932.0,Guy Arvely Dolsin\n\n[INST] <<SYS>>\nYou are ...,1932,...,4096,/opt/ml/model,1,1,inference.py,8,NaN,NaN,NaN,NaN
4,llama-2-70b-g5-48xlarge-1706460294,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1154.0,Yes\n\n[INST] <<SYS>>\nYou are an assistant f...,1154,...,4096,/opt/ml/model,1,1,inference.py,8,NaN,NaN,NaN,NaN


In [30]:
df_results = pd.merge(left=df_responses, right=df_endpoints, how='left', left_on='experiment_name', right_on='experiment_name')
df_results.head()

,endpoint_name,prompt,do_sample,temperature,top_p,top_k,max_new_tokens,truncate,completion,prompt_tokens,...,MAX_TOTAL_TOKENS,MODEL_CACHE_ROOT,SAGEMAKER_ENV,SAGEMAKER_MODEL_SERVER_WORKERS,SAGEMAKER_PROGRAM,SM_NUM_GPUS,HEALTH_CHECK_TIMOUT,INSTANCE_COUNT,MODEL_LOADING_TIMEOUT,NUMBER_OF_GPU
0,llama-2-70b-g5-48xlarge-1706460294,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,304.0,\n\n[INST] <<SYS>>\nYou are an assistant for q...,304,...,4096,/opt/ml/model,1,1,inference.py,8,NaN,NaN,NaN,NaN
1,llama-2-70b-g5-48xlarge-1706460294,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,980.0,\n\n[INST] <<SYS>>\n\n```\nPassage 1:\nWAGS At...,980,...,4096,/opt/ml/model,1,1,inference.py,8,NaN,NaN,NaN,NaN
2,llama-2-70b-g5-48xlarge-1706460294,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1339.0,\n\n[INST] <<SYS>>\nYou are an assistant for q...,1339,...,4096,/opt/ml/model,1,1,inference.py,8,NaN,NaN,NaN,NaN
3,llama-2-70b-g5-48xlarge-1706460294,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1932.0,Guy Arvely Dolsin\n\n[INST] <<SYS>>\nYou are ...,1932,...,4096,/opt/ml/model,1,1,inference.py,8,NaN,NaN,NaN,NaN
4,llama-2-70b-g5-48xlarge-1706460294,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1154.0,Yes\n\n[INST] <<SYS>>\nYou are an assistant f...,1154,...,4096,/opt/ml/model,1,1,inference.py,8,NaN,NaN,NaN,NaN


In [31]:
# Convert df_results to CSV and write to S3
csv_buffer = io.StringIO()
df_results.to_csv(csv_buffer, index=False)
csv_data_results = csv_buffer.getvalue()
results_file_name = config['results']['per_inference_request_file'].format(datetime=date_time)
results_s3_path = os.path.join(METRICS_DIR, results_file_name)
logger.info(f"results s3 path for per inference csv --> {results_s3_path}")
write_to_s3(csv_data_results, BUCKET_NAME, "", METRICS_DIR, results_file_name)
logger.info(f"saved results dataframe of shape={df_results.shape} in s3://{BUCKET_NAME}/{results_s3_path}")

[2024-01-28 13:22:52,767] p81084 {2449880785.py:7} INFO - results s3 path for per inference csv --> data/metrics/llama2-70b-g5-p4d-trt-v1/per_inference_request_results.csv


[2024-01-28 13:22:53,994] p81084 {2449880785.py:9} INFO - saved results dataframe of shape=(246, 32) in s3://fmbt2039/data/metrics/llama2-70b-g5-p4d-trt-v1/per_inference_request_results.csv


Data successfully written to s3://fmbt2039/data/metrics/llama2-70b-g5-p4d-trt-v1/per_inference_request_results.csv


In [32]:
df_metrics = pd.merge(left=df_metrics, right=df_endpoints, how='left', left_on='experiment_name', right_on='experiment_name')
df_metrics.head()

# Convert df_metrics to CSV and write to S3
csv_buffer = io.StringIO()
df_metrics.to_csv(csv_buffer, index=False)
csv_data_metrics = csv_buffer.getvalue()
metrics_file_name = config['results']['all_metrics_file'].format(datetime=date_time)
metrics_s3_path = os.path.join(METRICS_DIR, metrics_file_name)
logger.info(f"results s3 path for metrics csv --> {metrics_s3_path}")
write_to_s3(csv_data_metrics, BUCKET_NAME, "", METRICS_DIR, metrics_file_name)
logger.info(f"saved metrics results dataframe of shape={df_metrics.shape} in s3://{BUCKET_NAME}/{metrics_s3_path}")

[2024-01-28 13:22:54,090] p81084 {3262244809.py:10} INFO - results s3 path for metrics csv --> data/metrics/llama2-70b-g5-p4d-trt-v1/all_metrics.csv


[2024-01-28 13:22:54,424] p81084 {3262244809.py:12} INFO - saved metrics results dataframe of shape=(84, 34) in s3://fmbt2039/data/metrics/llama2-70b-g5-p4d-trt-v1/all_metrics.csv


Data successfully written to s3://fmbt2039/data/metrics/llama2-70b-g5-p4d-trt-v1/all_metrics.csv
